In [1]:
import optuna
import pickle
import torch
import os
import sys
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import yaml
import pandas as pd
from matplotlib.ticker import MultipleLocator
from scipy.optimize import linear_sum_assignment


# Add the root folder to Python path
root_folder = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_folder not in sys.path:
    sys.path.append(root_folder)

import lib.unet_nn as UNet
import lib.modified_aggregation as MA
from lib.modified_aggregation_clusterer import ModifiedAggregationClusterer
from lib.unet_clusterer import UNetClusterer
from lib.focal import FocalH
from lib.base_nn import Data
from lib import metrics# import count_clusters, count_labels, compute_score, total, separation_efficiency

os.chdir('/home/bjartur/workspace/python_focalh_clustering/') # Laptop and Desktop

# Study overview

In [2]:
study_file = "studies/study_dbscan_08092025_154943.pkl"
with open(study_file, "rb") as f:
    loaded_bundle = pickle.load(f)
#print(f"bundle: \n{loaded_bundle}")
#print()
#print(f"study: {loaded_bundle['study'].best_params}")
#print(loaded_bundle["timestamps"])
print(f"data: \n{loaded_bundle["data"]}")
print(f"its: {loaded_bundle["its"]}")


data: 
{'name': 'train_single_small', 'files': [{'path': 'data/train/TRAIN_E350_P1_N100.root', 'particles': 1, 'saturation': 4096, 'meta': {'source': 'mc', 'detector': 'prototype2'}}, {'path': 'data/train/TRAIN_E300_P1_N100.root', 'particles': 1, 'saturation': 4096, 'meta': {'source': 'mc', 'detector': 'prototype2'}}, {'path': 'data/train/TRAIN_E250_P1_N100.root', 'particles': 1, 'saturation': 4096, 'meta': {'source': 'mc', 'detector': 'prototype2'}}, {'path': 'data/train/TRAIN_E200_P1_N100.root', 'particles': 1, 'saturation': 4096, 'meta': {'source': 'mc', 'detector': 'prototype2'}}, {'path': 'data/train/TRAIN_E150_P1_N100.root', 'particles': 1, 'saturation': 4096, 'meta': {'source': 'mc', 'detector': 'prototype2'}}, {'path': 'data/train/TRAIN_E100_P1_N100.root', 'particles': 1, 'saturation': 4096, 'meta': {'source': 'mc', 'detector': 'prototype2'}}, {'path': 'data/train/TRAIN_E80_P1_N100.root', 'particles': 1, 'saturation': 4096, 'meta': {'source': 'mc', 'detector': 'prototype2'}}, {

In [3]:
loaded_bundle["timestamps"]["t_optimize_end"] - loaded_bundle["timestamps"]["t_optimize_start"]

25.44451642036438

In [4]:
def lst_test(par1=None, par2=None, par3=None):
    print(f"par1: {par1}")
    print(f"par2: {par2}")
    print(f"par3: {par3}")
lst = ["adsf", "bla"]
lst_test(*lst)

par1: adsf
par2: bla
par3: None


# Evaluate studies

In [5]:
ma_study_file = "studies/best/study_ma_24082025_133543.pkl"

cnn_study_file = "studies/study_cnn_24082025_132142.pkl"
cnn_model_file = "studies/model_cnn_24082025_132142.pt"

In [6]:
with open(ma_study_file, "rb") as f:
    loaded_bundle = pickle.load(f)
ma_study = loaded_bundle
ma = MA.ModifiedAggregation(ma_study["study"].best_params["seed"], ma_study["study"].best_params["agg"])
#ma = MA.ModifiedAggregation(1400, 0)
ma_study["exec"] = ma
ma_study["study"].best_params
ma_study

{'method': 'ma',
 'study': <optuna.study.study.Study at 0x725de910a150>,
 'config': {'analysis': {'type': 'standard',
   'files': [{'path': 'data/E150_P5_N1000.root',
     'particles': 5,
     'saturation': 4096,
     'meta': {'source': 'mc', 'detector': 'prototype2'}},
    {'path': 'data/E150_P3_N1000.root',
     'particles': 3,
     'saturation': 4096,
     'meta': {'source': 'mc', 'detector': 'prototype2'}}]}},
 'its': 100,
 'exec': <lib.modified_aggregation.ModifiedAggregation at 0x725deb49f350>}

In [7]:
with open(cnn_study_file, "rb") as f:
    loaded_bundle = pickle.load(f)
u = torch.load(cnn_model_file, weights_only=False)
cnn_study = loaded_bundle
cnn_study["exec"] = u
cnn_study["study"].best_params
cnn_study

{'method': 'cnn',
 'study': <optuna.study.study.Study at 0x725dea1d5c40>,
 'config': {'analysis': {'type': 'standard',
   'files': [{'path': 'data/E150_P5_N1000.root',
     'particles': 5,
     'saturation': 4096,
     'meta': {'source': 'mc', 'detector': 'prototype2'}},
    {'path': 'data/E150_P3_N1000.root',
     'particles': 3,
     'saturation': 4096,
     'meta': {'source': 'mc', 'detector': 'prototype2'}}]}},
 'its': 10,
 'exec': UNet(
   (e11): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (e12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (e21): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (e22): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (e31): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [8]:
def cluster_event(tfile, entry, study):
    """
    Just try clustering an event and view it.
    """
#    study["exec"]

    SAT = 4096
    tfile = ROOT.TFile(tfile, "READ")
    ttree = tfile.Get("EventsTree")
    ttree.GetEntry(entry)

    nplab = np.array(ttree.labels, dtype=np.int32)
    npfrac = np.array(ttree.fractions, dtype=np.float32)
    npvals = np.array(ttree.value, dtype=np.float32)
    valmask = npvals > SAT
    npvals[valmask] = SAT
    particles = len(set(nplab))
    
    dataloader = Data()
    npdlab = dataloader.get_major_labels(nplab, npfrac, particles)




    if study["method"] == "ma":
        iadj = np.load("../p2_sim_adj_map2.npy")
        adj = np.load("../p2_adj.npy")
        fig, ax = plt.subplots(ncols=2, figsize=(8,4))        
        foc = FocalH()
        foc.heatmap(npvals[iadj], npdlab[iadj], ax[0], SAT)
        clabels,_ = study["exec"].run(adj, npvals[iadj])
        foc.heatmap(npvals[iadj], clabels, ax[1], SAT)
        ax[0].set_title(f"Original. N particles: {count_labels(npdlab)}")
        ax[1].set_title(f"Clustered. N clusters: {count_clusters(clabels)}")

        fig.savefig(study["method"]+"_P"+str(particles)+"_EV"+str(entry))
            
    elif study["method"] == "cnn":
        iadj = np.load("../p2_sim_adj_map2.npy")
        adj = np.load("../p2_image_adj_21x21.npy")
        fig, ax = plt.subplots(ncols=3, figsize=(12,4))        
        foc = FocalH()
        foc.heatmap(npvals[iadj], npdlab[iadj], ax[0], SAT)
        ret, coms, dlabels, mapping = dataloader.read_ttree_event(ttree, entry)
        #target = dataloader.gaussian_class_activation_map(coms, 21, 21, 3)[0][0]

        x = study["exec"](ret)[0][0]
        vals = x.flatten().detach().numpy()
#        vals = vals / vals.max()
        ax[1].imshow(x.detach().numpy())

        seed = study["study"].best_params["seed"]
        agg = study["study"].best_params["agg"]
#        clusterizer = MA.ModifiedAggregation(seed,agg)
        clusterizer = MA.ModifiedAggregation(0.2,0)
        clabels,_ = clusterizer.run(adj,vals)

        clabels = dataloader.invert_labels(clabels, mapping, vals, npdlab.shape[0])
#        print(vals.max())
        foc.heatmap(npvals[iadj], clabels.astype(np.int32), ax[2], SAT)
        ax[0].set_title(f"Original. N particles: {count_labels(npdlab)}")
        ax[1].set_title(f"Smoothed showers")
        ax[2].set_title(f"Clustered. N clusters: {count_clusters(clabels)}")

        fig.savefig(study["method"]+"_P"+str(particles)+"_EV"+str(entry))

        

    
    

    
filename = "/home/bjartur/workspace/python_focalh_clustering/data/E300_P10_N1000.root"
entry = 400
cluster_event(filename, entry, cnn_study)
cluster_event(filename, entry, ma_study)

FileNotFoundError: [Errno 2] No such file or directory: '../p2_sim_adj_map2.npy'

# Sklearn

In [ ]:
hdbscan_study_file = "studies/study_hdbscan_03092025_192825.pkl"

with open(hdbscan_study_file, "rb") as f:
    loaded_bundle = pickle.load(f)
hdbscan_study = loaded_bundle
print(hdbscan_study["study"].best_params)
hdbscan_study

# Analyse study stability

Probably a good thing to, for each study, be able to show if the parameters seem to converge.

# Curtain test

In [ ]:
x_synth = np.zeros(21*21, dtype=np.float32).reshape(21,21)

fig,ax=plt.subplots(ncols=2, figsize=(10,5))

size=5
c1 = (9,9)
b1 = slice(c1[0],c1[0]+size), slice(c1[1],c1[1]+size)


c2 = (5,5)
b2 = slice(c2[0],c2[0]+size), slice(c2[1],c2[1]+size)


x_synth[b1] = 1
x_synth[b2] = 1

x_synth_tensor = torch.from_numpy(x_synth).unsqueeze(0).unsqueeze(0)
x_synth_pred = cnn_study["exec"](x_synth_tensor)
ax[0].imshow(x_synth)
ax[1].imshow(x_synth_pred[0][0].detach().numpy())

In [ ]:
fig,ax=plt.subplots(nrows=2,ncols=4, figsize=(10,5))

for i in range(4):
    i_synth = np.zeros(21*21, dtype=np.float32).reshape(21,21)
    origin = 4,4
    size = 2*(i+1)
    bi = slice(origin[0],origin[0]+size), slice(origin[1],origin[1]+size)
    i_synth[bi] = 1
    
    i_synth_tensor = torch.from_numpy(i_synth).unsqueeze(0).unsqueeze(0)
    i_synth_pred = cnn_study["exec"](i_synth_tensor)
    ax[0][i].imshow(i_synth)
    ax[1][i].imshow(i_synth_pred[0][0].detach().numpy())

fig.savefig("synth_blobs.png", bbox_inches="tight")

# Analyse the magic

Add masks over showers and see if it's able to intelligently reconstruct what is behind the mask (saturated area).

Call it curtain test.

# Analyse evaluation

In [ ]:
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "sans-serif",
    "font.sans-serif": ["DejaVu Sans"],
    "font.size" : 15,
    "legend.fontsize" : 12,
    "xtick.direction" : "in",
    "ytick.direction" : "in",
    "xtick.major.size" : 5.0,
    "xtick.minor.size" : 3.0,
    "ytick.major.size" : 5.0,
    "ytick.minor.size" : 3.0,
    "axes.linewidth" : 0.8,
    "legend.handlelength" : 2,
})

def plot_discrete(x,y,label=""):
    unq = np.unique(x)
    muy = np.zeros_like(unq)
    mux = np.zeros_like(unq)
    sigmay = np.zeros_like(unq)
    sigmax = np.zeros_like(unq)
    for i in range(len(unq)):
        mask = x == unq[i]
        muy[i] = y[mask].mean()
        mux[i] = x[mask].mean()
        sigmay[i] = y[mask].std()/np.sqrt(len(y[mask]))
        sigmax[i] = x[mask].std()/np.sqrt(len(x[mask]))

    plt.errorbar(mux,muy,sigmay,marker=".",linestyle="", label=label)
    plt.ylim(0,2)
    plt.legend()


def plot_performance_particle_num(df, x_col, y_col, marker_shape="^", linestyle="", color="", ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    grp = df.groupby(x_col)[y_col]
    mu_x = grp.mean().index
    mu_y = grp.mean().values
    sigma_y = grp.sem().values
    if color == "":
        ax.errorbar(mu_x, mu_y, yerr=sigma_y, marker=marker_shape, linestyle=linestyle, capsize=5)
    else:
        ax.errorbar(mu_x, mu_y, yerr=sigma_y, marker=marker_shape, linestyle=linestyle, color=color, capsize=5)
    #ax.plot(mu_x, mu_y)


def plot_chunks(x, y, Nbins, label, marker_shape=".", color="", ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    bins = np.linspace(min(x), max(x), Nbins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    mu_y = np.zeros(Nbins)
    sigma_y = np.zeros(Nbins)
    
    for i in range(Nbins):
        mask = (x >= bins[i]) & (x < bins[i+1])
        if np.any(mask):
            mu_y[i] = np.mean(y[mask])
            sigma_y[i] = np.std(y[mask]) / np.sqrt(np.sum(mask))

    if color == "":
        ax.errorbar(bin_centers, mu_y, yerr=sigma_y, marker=marker_shape, linestyle='solid', capsize=5)
    else:
        ax.errorbar(bin_centers, mu_y, yerr=sigma_y, marker=marker_shape, linestyle='solid', color=color, capsize=5)




def format_with_suffix(num):
    if num >= 1e9:
        return f"{num / 1e9:.1f}B"
    elif num >= 1e6:
        return f"{num / 1e6:.1f}M"
    elif num >= 1e3:
        return f"{num / 1e3:.1f}K"
    else:
        return str(num)


#plot_discrete(count, score)

In [ ]:
#ma_eval_file = "evaluation/eval_ma_01092025_053548.pkl"
#ma_eval_file = "evaluation/eval_ma_07092025_103817.pkl"
#ma_eval_file = "evaluation/eval_ma_06092025_102441.pkl"
#ma_eval_file = "evaluation/eval_ma_08092025_092411.pkl"
#ma_eval_file = "evaluation/eval_ma_08092025_200714.pkl" # train_single_small
#ma_eval_file = "evaluation/eval_ma_small_many_sep_13092025_173010.pkl" # train_small_100 sep_eff
#ma_eval_file = "evaluation/eval_ma_tiny_many_sep_13092025_195950.pkl" # train_tiny_1000 sep_eff
#ma_eval_file = "evaluation/eval_ma_tiny_single_sep_13092025_204113.pkl" # train_tiny_single_1000 sep_eff
ma_eval_file = "evaluation/eval_ma_two_15092025_130928.pkl" # train_tiny_single_1000 sep_eff eval_two






#cnn_eval_file = "evaluation/eval_cnn_02092025_060217.pkl"
#cnn_eval_file = "evaluation/eval_cnn_02092025_104200.pkl"
#cnn_eval_file = "evaluation/eval_cnn_07092025_094926.pkl"
cnn_eval_file = "evaluation/eval_cnn_07092025_102542.pkl"



#hdbscan_eval_file = "evaluation/eval_hdbscan_03092025_201034.pkl"
#hdbscan_eval_file = "evaluation/eval_hdbscan_07092025_103658.pkl"
#hdbscan_eval_file = "evaluation/eval_hdbscan_05092025_113309.pkl"
hdbscan_eval_file = "evaluation/eval_hdbscan_08092025_182644.pkl" # train_tiny
#hdbscan_eval_file = "evaluation/eval_hdbscan_08092025_194252.pkl" # train_tiny smaller multiply



dbscan_eval_file = "evaluation/eval_dbscan_08092025_162047.pkl"


with open(cnn_eval_file, "rb") as f:
    loaded_bundle = pickle.load(f)
cnn_eval = loaded_bundle


with open(ma_eval_file, "rb") as f:
    loaded_bundle = pickle.load(f)
ma_eval = loaded_bundle


with open(hdbscan_eval_file, "rb") as f:
    loaded_bundle = pickle.load(f)
hdbscan_eval = loaded_bundle


with open(dbscan_eval_file, "rb") as f:
    loaded_bundle = pickle.load(f)
dbscan_eval = loaded_bundle


#cnn_eval["method"]
#cnn_eval["eval"]


#hdbscan_eval["data"]

In [ ]:
ma_eval["eval"]["separation"][:,0]

In [ ]:
def test_separation(d, e):
    energies = d["eval"]["energy"][e]
    labels = d["eval"]["labels"][e]
    values = d["eval"]["values"][e]
    tags = d["eval"]["tags"][e]
    foc = FocalH()
    iadj = np.load("p2_sim_adj_map2.npy")
    fig,ax=plt.subplots(ncols=2)
    foc.heatmap(values[iadj], labels[iadj],ax[0])
    foc.heatmap(values[iadj], tags[iadj],ax[1]) # EVAL IS WRONGLY NOT USING IADJ
    separated = metrics.resolved(tags[iadj], labels[iadj], values[iadj], energies, 298.2, 7570, 0.97, 0.00, 0.08)
#    separated = metrics.resolved(tags[iadj], labels[iadj], values[iadj], energies, 181.52, -3272.28, 0.97, 0.00, 0.08)
#    separated = metrics.resolved(tags[iadj], labels[iadj], values[iadj], energies, 298.2, 7570, 1.18, 0.00, 0.09)
    print(f"Resolved: {separated[0]} out of {separated[1]}")
    print(f"Efficiency: {d["eval"]["efficiency"][e]}")
#    resolved(tags[iadj], labels[iadj], values[iadj], energies, 181.52, -3272, 0.97, 0.00, 0.08)


    # 150
test_separation(ma_eval, 1000)

In [ ]:
300 * np.sqrt(  (0.97**2)/300 + (0.00**2)/(300**2) + 0.09**2)
#sigma_E = np.sqrt((0.97**2)/300 + (b**2)/(E**2) + c**2)

metrics.energy_resolution(300, 0.97, 0.00, 0.09)


In [ ]:
0.1*300

In [ ]:
metrics.reconstruct_energy(117446,298.2, 7570)

In [ ]:
import importlib
importlib.reload(metrics)

res = metrics.separation_efficiency_opt(ma_eval["eval"]["tags"], ma_eval["eval"]["labels"], ma_eval["eval"]["values"], ma_eval["eval"]["energy"])
#from lib.metrics import count_clusters, count_labels, compute_score, total, separation_efficiency


In [ ]:
res.mean()

In [ ]:
def dist(coms):
    print(len(coms))
    dists = [None]*len(coms)
    for i in range(len(coms)):
        diff = coms[i][:, np.newaxis, :] - coms[i][np.newaxis, :, :]
        dist = np.sqrt((diff ** 2).sum(axis=-1))
        upper_tri = dist[np.triu_indices_from(dist, k=1)]
        dists[i] = upper_tri.mean()
    return dists


def to_dataframe(d):
#    print(d["method"])
    df = pd.DataFrame({
        "efficiency": d["eval"]["efficiency"],
        "vmeasure": d["eval"]["vmeasure"],
#        "vmeasure_weighted": d["vmeasure_weighted"],
        "coverage": d["eval"]["coverage"],
        "particles": d["eval"]["particles"],
        "avg_energy": d["eval"]["avg_energy"],
        "coms_dists" : dist(d["eval"]["coms"]),
        "separation_resolved" : d["eval"]["separation"][:,0],
        "separation_total" : d["eval"]["separation"][:,1],
                         })
    return df



df_ma = to_dataframe(ma_eval)
#df_cnn = to_dataframe(cnn_eval)
#df_hdbscan = to_dataframe(hdbscan_eval)
#df_dbscan = to_dataframe(dbscan_eval)

In [ ]:

plt.plot(df_ma["efficiency"])
df_ma["efficiency"].mean()

In [ ]:

fig,ax = plt.subplots()
plot_performance_particle_num(df_ma, "avg_energy", "efficiency", marker_shape="^", linestyle="dashed", ax=ax)
plot_performance_particle_num(df_cnn, "avg_energy", "efficiency", marker_shape="s", linestyle="dashed", ax=ax)
plot_performance_particle_num(df_hdbscan, "avg_energy", "efficiency", marker_shape="o", linestyle="dashed", ax=ax)
plot_performance_particle_num(df_dbscan, "avg_energy", "efficiency", marker_shape="o", linestyle="dashed", ax=ax)
#ax.set_xticks([1,2,3,4,5,6,7,8,9,10], ["1","2","3","4","5","6","7","8","9","10"])
ax.set_xticks([0,50,100,150,200,250,300,350], ["0","50","100","150","200","250","300","350"])
#ax.set_xticks([1,2], ["1","2"])
ax.set_ylim(0,3)
ax.xaxis.set_minor_locator(MultipleLocator(10))
#ax.yaxis.set_minor_locator(MultipleLocator(0.2))
ax.yaxis.set_ticks_position("both")
ax.xaxis.set_ticks_position("both")
ax.set_xlabel("Energy [GeV]")
ax.set_ylabel("Efficiency [$N_{clusters}/N_{particles}$]")
ax.axhline(1, color="grey", linestyle="dotted")
ax.scatter([],[], label="Modified Aggregation", marker="^")
ax.scatter([],[], label="CNN+Modified Aggregation", marker="s")
ax.scatter([],[], label="HDBSCAN", marker="s")
ax.scatter([],[], label="DBSCAN", marker="s")
ax.legend(framealpha=0)

#ax.text(0.00, 1.05, "FoCal-H Prototype 2", transform=plt.gca().transAxes, ha="left", va="center")
#ax.text(1.00, 1.05, f"${len(df_ma):.0E}$ MC Events", transform=plt.gca().transAxes, ha="right", va="center")


ax.text(0.03, 0.93, "FoCal-H", transform=plt.gca().transAxes, ha="left", va="center")
ax.text(0.03, 0.86, "Prototype 2 Monte-Carlo", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)
ax.text(0.03, 0.80, f"1-10 particles", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)
ax.text(0.03, 0.74, f"{format_with_suffix(len(df_ma))} events", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)


#ax.grid(True)
fig.savefig("eval_eff_energy_many.png", bbox_inches="tight")

In [ ]:
ma_single = df_ma["particles"] == 1
cnn_single = df_cnn["particles"] == 1
hdbscan_single = df_hdbscan["particles"] == 1
dbscan_single = df_dbscan["particles"] == 1

fig,ax = plt.subplots()
plot_performance_particle_num(df_ma[ma_single], "avg_energy", "efficiency", marker_shape="^", linestyle="dashed", ax=ax)
plot_performance_particle_num(df_cnn[cnn_single], "avg_energy", "efficiency", marker_shape="s", linestyle="dashed", ax=ax)
plot_performance_particle_num(df_hdbscan[hdbscan_single], "avg_energy", "efficiency", marker_shape="o", linestyle="dashed", ax=ax)
plot_performance_particle_num(df_dbscan, "avg_energy", "efficiency", marker_shape="o", linestyle="dashed", ax=ax)
#ax.set_xticks([1,2,3,4,5,6,7,8,9,10], ["1","2","3","4","5","6","7","8","9","10"])
ax.set_xticks([0,50,100,150,200,250,300,350], ["0","50","100","150","200","250","300","350"])
#ax.set_xticks([1,2], ["1","2"])
ax.set_ylim(0,4)
ax.xaxis.set_minor_locator(MultipleLocator(10))
#ax.yaxis.set_minor_locator(MultipleLocator(0.2))
ax.yaxis.set_ticks_position("both")
ax.xaxis.set_ticks_position("both")
ax.set_xlabel("Energy [GeV]")
ax.set_ylabel("Efficiency [$N_{clusters}/N_{particles}$]")
ax.axhline(1, color="grey", linestyle="dotted")
ax.scatter([],[], label="Modified Aggregation", marker="^")
ax.scatter([],[], label="CNN+Modified Aggregation", marker="s")
ax.scatter([],[], label="HDBSCAN", marker="o")
ax.scatter([],[], label="DBSCAN", marker="o")
ax.legend(framealpha=0)

ax.text(0.03, 0.93, "FoCal-H", transform=plt.gca().transAxes, ha="left", va="center")
ax.text(0.03, 0.86, "Prototype 2 Monte-Carlo", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)
ax.text(0.03, 0.80, f"Single particles", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)
ax.text(0.03, 0.74, f"{format_with_suffix(len(df_ma[ma_single]))} events", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)

fig.savefig("eval_eff_energy_single.png", bbox_inches="tight")

In [ ]:
fig,ax = plt.subplots()
plot_performance_particle_num(df_ma, "particles", "efficiency", marker_shape="^", linestyle="dashed", ax=ax)
plot_performance_particle_num(df_cnn, "particles", "efficiency", marker_shape="s", linestyle="dashed", ax=ax)
plot_performance_particle_num(df_hdbscan, "particles", "efficiency", marker_shape="o", linestyle="dashed", ax=ax)
ax.set_xticks([1,2,3,4,5,6,7,8,9,10], ["1","2","3","4","5","6","7","8","9","10"])
#ax.set_xticks([0,50,100,150,200,250,300,350], ["0","50","100","150","200","250","300","350"])
#ax.set_xticks([1,2], ["1","2"])
ax.set_ylim(0,3)
ax.xaxis.set_minor_locator(MultipleLocator(10))
#ax.yaxis.set_minor_locator(MultipleLocator(0.2))
ax.yaxis.set_ticks_position("both")
ax.xaxis.set_ticks_position("both")
ax.set_xlabel("Particles")
ax.set_ylabel("Efficiency [$N_{clusters}/N_{particles}$]")
ax.axhline(1, color="grey", linestyle="dotted")
ax.scatter([],[], label="Modified Aggregation", marker="^")
ax.scatter([],[], label="CNN+Modified Aggregation", marker="s")
ax.scatter([],[], label="HDBSCAN", marker="o")
ax.legend(framealpha=0)

#ax.text(0.00, 1.05, "FoCal-H Prototype 2", transform=plt.gca().transAxes, ha="left", va="center")
#ax.text(1.00, 1.05, f"${len(df_ma):.0E}$ MC Events", transform=plt.gca().transAxes, ha="right", va="center")

ax.text(0.03, 0.93, "FoCal-H", transform=plt.gca().transAxes, ha="left", va="center")
ax.text(0.03, 0.86, "Prototype 2 Monte-Carlo", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)
ax.text(0.03, 0.80, f"Energy: 20-350 GeV", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)
ax.text(0.03, 0.74, f"{format_with_suffix(len(df_ma[ma_single]))} events", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)



#ax.grid(True)
fig.savefig("eval_eff_particles.png", bbox_inches="tight")

In [ ]:
ma_two = df_ma["particles"] == 2
cnn_two = df_cnn["particles"] == 2
hdbscan_two = df_hdbscan["particles"] == 2
bins=10
fig,ax = plt.subplots()
#plot_performance_particle_num(df_ma[ma_two], "coms_dists", "vmeasure", marker_shape="^", linestyle="dashed", ax=ax)
plot_chunks(df_ma[ma_two]["coms_dists"], df_ma[ma_two]["efficiency"], bins, "Modified Aggregation", "o", "", ax)
plot_chunks(df_cnn[cnn_two]["coms_dists"], df_cnn[cnn_two]["efficiency"], bins, "CNN+Modified Aggregation", "o", "", ax)
plot_chunks(df_hdbscan[hdbscan_two]["coms_dists"], df_hdbscan[hdbscan_two]["efficiency"], bins, "HDBSCAN", "o", "", ax)

ax.set_xticks(
    [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    ["0", "2", "4", "6", "8", "10", "12", "14", "16", "18", "20"])
ax.set_ylim(0,2)
ax.xaxis.set_minor_locator(MultipleLocator(1))
#ax.yaxis.set_minor_locator(MultipleLocator(0.2))
ax.yaxis.set_ticks_position("both")
ax.xaxis.set_ticks_position("both")
ax.set_xlabel("Center-of-mass distance [cm]")
ax.set_ylabel("Efficiency [$N_{clusters}/N_{particles}$]")

ax.scatter([],[], label="Modified Aggregation", marker="^")
ax.scatter([],[], label="CNN+Modified Aggregation", marker="s")
ax.scatter([],[], label="HDBSCAN", marker="o")
ax.legend(framealpha=0)
ax.axhline(1, color="grey", linestyle="dotted")

ax.text(0.03, 0.93, "FoCal-H", transform=plt.gca().transAxes, ha="left", va="center")
ax.text(0.03, 0.86, "Prototype 2 Monte-Carlo", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)
ax.text(0.03, 0.80, f"Energy: 20-350 GeV", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)
ax.text(0.03, 0.74, f"2-particle events", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)
ax.text(0.03, 0.68, f"{format_with_suffix(len(df_ma[ma_two]))} events", transform=plt.gca().transAxes, ha="left", va="center", fontsize=10)



fig.savefig("eval_eff_com.png", bbox_inches="tight")

In [ ]:
def plot_chunks(x, y, Nbins, label, marker_shape=".", color="", ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    bins = np.linspace(min(x), max(x), Nbins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    mu_y = np.zeros(Nbins)
    sigma_y = np.zeros(Nbins)
    
    for i in range(Nbins):
        mask = (x >= bins[i]) & (x < bins[i+1])
        if np.any(mask):
            mu_y[i] = np.mean(y[mask])
            sigma_y[i] = np.std(y[mask]) / np.sqrt(np.sum(mask))

    if color == "":
        ax.errorbar(bin_centers, mu_y, yerr=sigma_y, marker=marker_shape, linestyle='solid', capsize=5)
    else:
        ax.errorbar(bin_centers, mu_y, yerr=sigma_y, marker=marker_shape, linestyle='solid', color=color, capsize=5)


def plot_separation(df,Nbins,ax):

    x = df["coms_dists"]
    bins = np.linspace(min(x), max(x), Nbins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    mu_y = np.zeros(Nbins)
    sigma_y = np.zeros(Nbins)


    for i in range(Nbins):
        mask = (x >= bins[i]) & (x < bins[i+1])
        if np.any(mask):
            mu_y[i] = df[mask]["separation_resolved"].sum() / df[mask]["separation_total"].sum()
            print(df[mask]["separation_resolved"].sum())
            #sigma_y[i] = #np.std(y[mask]) / np.sqrt(np.sum(mask))
    
    print(df["separation_resolved"].mean())
    ax.errorbar(bin_centers, mu_y, yerr=sigma_y, linestyle='solid', capsize=5)


    pass

two_mask = df_ma["particles"] == 2
en_mask = df_ma["avg_energy"] == df_ma["avg_energy"]
eff_mask = df_ma["efficiency"] == 1
fig,ax = plt.subplots()

#plot_chunks(df_ma[ma_two]["coms_dists"], df_ma[ma_two]["efficiency"], bins, "Modified Aggregation", "o", "", ax)

plot_separation(df_ma[two_mask & eff_mask],10,ax)


In [ ]:
df_ma[two_mask]

In [ ]:
df_cnn[cnn_two]

In [ ]:
from plotly.io import show

In [ ]:
print(ma_eval["study"].best_params)
fig = optuna.visualization.plot_param_importances(ma_eval["study"])
show(fig)

In [ ]:
fig = optuna.visualization.plot_optimization_history(cnn_eval["study"]["study"])
show(fig)